In [ ]:
# 这里的equal process是针对于已经分词后的文件处理的
# 1.创建equal_dict，用来替换的字典
# 2.使用'title_split'分段，发现equal_dict中的key属性即在title中进行替换
# 3.'拉链'分为'裤拉链' '鞋拉链' '拉链'，'系带'分为'裤系带' '鞋系带' '系带'
# 4.同时重新处理title为统一大写，只需要将替换后的'title_split'合并即可
# 5.重新生成新的合并后的processed_word_dict，并重新生成'vocab_split'分段


In [2]:
from tqdm import tqdm 
import json
import itertools

In [3]:
equal_dict = {'半高领': '高领',
 '立领': '高领',
 '可脱卸帽': '连帽',
 '衬衫领': '翻领',
 'POLO领': '翻领',
 '方领': '翻领',
 '娃娃领': '翻领',
 '荷叶领': '翻领',
 '五分袖': '短袖',
 '九分袖': '长袖',
 '超短款': '短款',
 '常规款': '短款',
 '超长款': '长款',
 '标准型': '修身型',
 '超短裙': '短裙',
 '中长裙': '中裙', 
 'O型裤': '哈伦裤',
 '灯笼裤': '哈伦裤',
 '锥形裤': '哈伦裤',
 '铅笔裤': '直筒裤',
 '小脚裤': '直筒裤',
 '微喇裤': '喇叭裤',
 '九分裤': '长裤',
 '套筒': '一脚蹬',
 '套脚': '一脚蹬',
 '中帮': '高帮'}
# 替换的工作要额外做一个，中长款替换为中款！

In [43]:
# 加载原始的属性字典
def load_attr_dict(file):
    # 读取属性字典
    with open(file, 'r') as f:
        attr_dict = {}
        for attr, attrval_list in json.load(f).items():
            attrval_list = list(map(lambda x: x.split('='), attrval_list))
            attr_dict[attr] = list(itertools.chain.from_iterable(attrval_list))
    return attr_dict

# load attribute dict
attr_dict_file = "../../data/original_data/attr_to_attrvals.json"
attr_dict = load_attr_dict(attr_dict_file)

# 相等替换
for query, attrs in attr_dict.items():
    attrs = attrs.copy()
    for i, attr in enumerate(attrs):
        if attr in equal_dict:
            attr_dict[query].remove(attr)
            
# 特殊的几个属性替换
for query, attrs in attr_dict.items():
    attrs = attrs.copy()
    for i, attr in enumerate(attrs):
        if query=='衣长' and attr=='中长款':
            attr_dict[query][i] = '中款'
        if query=='裤门襟' and attr=='拉链':
            attr_dict[query][i] = '拉链裤'
        if query=='裤门襟' and attr=='系带':
            attr_dict[query][i] = '系带裤'
        if query=='裤门襟' and attr=='松紧':
            attr_dict[query][i] = '松紧裤'
        if query=='闭合方式' and attr=='拉链':
            attr_dict[query][i] = '拉链鞋'
        if query=='闭合方式' and attr=='系带':
            attr_dict[query][i] = '系带鞋'

# 保存新的属性字典
attr_save_file = '../../data/equal_processed_data/attr_to_attrvals.json'
with open(attr_save_file, 'w') as f:
    json.dump(attr_dict, f, ensure_ascii=False, indent=4)
            

In [44]:
attr_file = '../../data/equal_processed_data/attr_to_attrvals.json'
with open(attr_file, 'r') as f:
    attr_dict = json.load(f)

In [ ]:
# [fine] 移除年份，统一大写，替换相等属性，替换特殊属性
fine_data = '../../data/original_data/train_fine.txt'
new_fine_data = '../../data/equal_processed_data/fine50000.txt'

rets = []
delete_list = ['2017年','2018年','2019年','2020年','2021年','2022年']
i = 0
with open(fine_data, 'r') as f:
    for i, data in enumerate(tqdm(f)):
        data = json.loads(data)
        title = data['title']
        title = title.upper() # 字母统一为大写
        for delete_item in delete_list:
            title = title.replace(delete_item, '')
        data['title'] = title
        
        rets.append(json.dumps(data, ensure_ascii=False)+'\n')
        
        # if i>500:
        #     break
        # i += 1
          
with open(new_fine_data, 'w') as f:
    f.writelines(rets)

In [14]:
# fine 
file = '../../data/split_word/fine45000.txt'
save_file = '../../data/final_processed_data/fine45000.txt'

word_dict = {}
rets = []
with open(file, 'r') as f:
    for i, line in enumerate(tqdm(f)):
        item = json.loads(line)
        title = item['title']
        title_split = item['title_split']
        
        # 替换title_split和title
        for n, word in enumerate(title_split):
            if word in equal_dict: # 等号替换
                title_split[n] = equal_dict[word]
            elif word in ['拉链', '系带'] and '裤' in title: # 拉链系带替换
                title_split[n] = ''.join(['裤', word])
            elif word in ['拉链', '系带'] and '鞋' in title: # 拉链系带替换
                title_split[n] = ''.join(['鞋', word])
        new_title = ''.join(title_split)
        item['title'] = new_title
        item['title_split'] = title_split
        
        # 替换key_attr
        key_attr = item['key_attr']
        for query, attr in key_attr.items():
            if attr in equal_dict:
                key_attr[query] = equal_dict[attr]
                
            
        # 统计word_dict
        for word in title_split:
            if word in word_dict:
                word_dict[word] += 1
            else:
                word_dict[word] = 1
        
        rets.append(json.dumps(item, ensure_ascii=False)+'\n')
        
        # if i>500:
        #     break
        # i += 1
        
with open(save_file, 'w') as f:
    f.writelines(rets)

3304it [00:03, 889.59it/s]


KeyboardInterrupt: 

In [ ]:
# coarse
file = '../../data/split_word/coarse89588.txt'
save_file = '../../data/final_processed_data/coarse89588.txt'

word_dict = {}
rets = []
with open(file, 'r') as f:
    for i, line in enumerate(tqdm(f)):
        item = json.loads(line)
        title = item['title']
        title_split = item['title_split']
        
        # 替换title_split和title
        for n, word in enumerate(title_split):
            if word in equal_dict: # 等号替换
                title_split[n] = equal_dict[word]
            elif word in ['拉链', '系带'] and '裤' in title: # 拉链系带替换
                title_split[n] = ''.join(['裤', word])
            elif word in ['拉链', '系带'] and '鞋' in title: # 拉链系带替换
                title_split[n] = ''.join(['鞋', word])
        new_title = ''.join(title_split)
        item['title'] = new_title
        item['title_split'] = title_split
        
        # 替换key_attr
        key_attr = item['key_attr']
        for query, attr in key_attr.items():
            if attr in equal_dict:
                key_attr[query] = equal_dict[attr]
                
            
        # 统计word_dict
        for word in title_split:
            if word in word_dict:
                word_dict[word] += 1
            else:
                word_dict[word] = 1
        
        rets.append(json.dumps(item, ensure_ascii=False)+'\n')
        
        # if i>500:
        #     break
        # i += 1
        
with open(save_file, 'w') as f:
    f.writelines(rets)

In [18]:
word_dict

{'春季': 50,
 '喇叭裤': 1,
 '牛仔裤': 12,
 '蓝色': 24,
 '常规': 100,
 '厚度': 100,
 '长裤': 64,
 '女装': 156,
 '浅咖': 1,
 '修身型': 156,
 '套头': 44,
 '针织衫': 18,
 '宽松型': 62,
 '牛仔蓝': 1,
 '高领': 36,
 '长袖': 112,
 '外套': 14,
 '纯色': 88,
 '秋季': 117,
 '加厚': 54,
 '灰色': 39,
 '童装': 46,
 '衬衫': 15,
 '短款': 93,
 '休闲': 34,
 '百搭': 22,
 '男士': 69,
 '棉服': 26,
 '拉链': 43,
 '中长款': 63,
 '翻领': 48,
 '男装': 149,
 '微弹': 16,
 '黑色': 132,
 '冬季': 92,
 '裤系带': 7,
 '字母': 1,
 '加绒': 26,
 '裤': 12,
 '墨绿': 2,
 '夏季': 39,
 'T恤': 35,
 '短袖': 33,
 '薄款': 28,
 '白色': 32,
 '正装裤': 6,
 '直筒裤': 34,
 '长款': 9,
 '女士': 76,
 '风衣': 16,
 '单排扣': 14,
 '卡其色': 7,
 '连帽': 46,
 '印花': 7,
 '儿童': 17,
 '酒红色': 6,
 '仿皮': 16,
 '皮衣': 30,
 '暗扣': 2,
 '羊毛衫': 13,
 'V领': 17,
 '鞋系带': 58,
 '运动': 14,
 '帆布鞋': 14,
 '低帮': 39,
 '轻便': 6,
 '卫衣': 19,
 '圆领': 46,
 '真皮': 12,
 '透气': 15,
 '登山鞋': 10,
 '高帮': 37,
 '休闲裤': 24,
 '毛呢大衣': 23,
 '西装领': 30,
 '松紧': 2,
 'POLO衫': 14,
 '条纹': 4,
 '黑白': 4,
 '一脚蹬': 5,
 '户外': 21,
 '工装鞋': 15,
 '绿色': 6,
 '毛衣': 6,
 '中裙': 2,
 '连衣裙': 4,
 '藏蓝色': 1,
 '浅咖色': 1,
 '藏青': 4,
 '牛津布': 1